## Imports

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

from datetime import datetime

In [2]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

## EMA Workbench Setup

In [3]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(7)

Via the `dike_model` variable, we can access scenarios (uncertainties), policies (levers), and our problem formulation (outcomes).

We want to do a run where we analyse a specific, simple, relevant set of policies across a large variety of scenarios based on the outcomes of interest we have deemed relevant for our Client. We seek to produce many data points (each the output of a single experiment, which is defined by the combination of a specific policy under a specific scenario) and compare them to see how these potential policies perform in a broad set of future scenarios.

## Defining Specific Policies

### General remarks:
1. In this early experimentation, we apply all levers at timestamp 0, so at the beginning of the 200years of the simulation. They can also be applied  after 67 or 133 years.
2. In this early experimentation, we are ignoring the policy lever "EWS_DaysToThreat." As analysts on behalf of the Dike Ring as a region, we are more interested in how infrastructural changes can be implemented to protect the region. We are opposed to evacuation as a policy on its own, because this does nothing to protect our Client's constituents' land. In later analysis, it may be worth testing various evacuation levers and effectively treating them as scenario variables.
3. In this early experimentation, we will only study policies that directly impact our Client's region. More can be added for later analysis of policies that the Rijkswaterstaat seems to prefer.

In [4]:
def get_do_nothing_dict():
    return {l.name: 0 for l in dike_model.levers}

policies = [
    Policy(
        "policy 1",
        # Big RfR in most upstream location (based on the map in ring 1 (non-organic farming)) -
        # our client would prefer this, as they don't care about non-organic farming and it should 
        # significantly decrease flooding of ring 4 (town Gorssel).
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1}
            # **{"0_RfR 0": 1, "0_RfR 1": 1, "0_RfR 2": 1, "A.1_DikeIncrease 0": 5} - 
            # this was original "policy 1" - since the course manual states: (for RfR 
            #"Once activated, the project remains active.", I think ""0_RfR 1": 1, "0_RfR 2": 1" 
            # are redundant - but we should evaluate this.
        )
    ),
    Policy(
        "policy 2",
        # Second best alternative of RfR from ring 4 perspective. Some RfR made upstream, 
        # some downstream from them, but none on their area. We choose this to have some comparison for policy 1.
        **dict(
            get_do_nothing_dict(),
            **{"3_RfR 0": 1}
        )
    ),
    Policy(
        "policy 3",
        # Only hightening dikes in ring 4 by the middle height of 0.5m (5 decimeters).
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 5}
        )
    ),
    Policy(
        "policy 4",
        # Only hightening dikes in ring 4 by maximum height of 1m for comparison with policy 3.
        **dict(
            get_do_nothing_dict(),
            **{"A.4_DikeIncrease 0": 10}
        )
    ),
    Policy(
        "policy 5",
        # policy 1 + policy 3 - trade-off between costs of projects and max safety. Note that making more RfR 
        # (outside of ring 4 though, as they don't want to give up land) 
        # and hightening the dikes in ring 4 by more than 0.5m would increase ring 4 safety even further, but
        # is unrealistic with limiteed budget and stakeholders' interests.
        **dict(
            get_do_nothing_dict(),
            **{"0_RfR 0": 1, "A.4_DikeIncrease 0": 5}
        )
    ),
]

## Running Experiments

In [5]:
# pass the policies list to EMA workbench experiment runs
n_scenarios = 200
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios, policies)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 200 scenarios * 5 policies * 1 model(s) = 1000 experiments
100%|██████████████████████████████████████| 1000/1000 [03:12<00:00,  5.19it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


## Capturing Results

Detailed analysis of the results will be done in a separate notebook, so this one can exclusively be used for generating experimental results (since this will be an iterative process). An initial view of the results tables is shown below.

In [6]:
CURRENT_DATE = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')

In [7]:
experiments, outcomes = results

In [8]:
experiments_df = pd.DataFrame(experiments)
experiments_df

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.3_DikeIncrease 2,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,scenario,policy,model
0,70,241.643948,10.0,0.170110,146.452423,10.0,0.459859,54.557065,1.0,0.305443,...,0,0,0,0,0,0,0,0,policy 1,dikesnet
1,78,101.962476,1.5,0.260697,90.617465,1.5,0.204544,50.659147,1.5,0.290542,...,0,0,0,0,0,0,0,1,policy 1,dikesnet
2,45,112.523857,10.0,0.972273,289.154892,1.5,0.645199,315.847914,1.5,0.738454,...,0,0,0,0,0,0,0,2,policy 1,dikesnet
3,119,211.231073,1.5,0.195967,324.942780,1.0,0.091564,297.250535,10.0,0.080798,...,0,0,0,0,0,0,0,3,policy 1,dikesnet
4,129,73.118072,1.5,0.416433,307.773944,10.0,0.087428,37.517170,1.0,0.598444,...,0,0,0,0,0,0,0,4,policy 1,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,47,123.290623,1.5,0.360736,216.000643,1.0,0.301859,152.905948,10.0,0.122215,...,0,5,0,0,0,0,0,195,policy 5,dikesnet
996,72,84.110012,1.5,0.414538,263.633154,10.0,0.155070,333.466384,1.0,0.274567,...,0,5,0,0,0,0,0,196,policy 5,dikesnet
997,100,247.508444,10.0,0.627586,235.816933,1.0,0.731786,285.877296,10.0,0.343657,...,0,5,0,0,0,0,0,197,policy 5,dikesnet
998,77,279.364305,1.5,0.306045,147.207557,1.0,0.886717,245.817251,10.0,0.750266,...,0,5,0,0,0,0,0,198,policy 5,dikesnet


In [9]:
# TODO: make this work with problem formulation 6
outcomes_df = pd.DataFrame(outcomes)
outcomes_df

,A.1 Expected Annual Damage,A.1 Expected Number of Deaths,A.2 Expected Annual Damage,A.2 Expected Number of Deaths,A.4 Expected Annual Damage,A.4 Expected Number of Deaths,A.5 Expected Annual Damage,A.5 Expected Number of Deaths,Total Infrastructure Costs
0,7.367290e+08,0.494021,7.920079e+06,0.007721,0.000000e+00,0.000000,1.006875e+09,0.902142,8.460000e+07
1,7.585044e+07,0.054368,2.625348e+08,0.266427,0.000000e+00,0.000000,0.000000e+00,0.000000,8.460000e+07
2,0.000000e+00,0.000000,3.854790e+07,0.039665,1.074579e+08,0.051536,0.000000e+00,0.000000,8.460000e+07
3,2.915187e+08,0.255663,5.284940e+08,0.636090,0.000000e+00,0.000000,0.000000e+00,0.000000,8.460000e+07
4,0.000000e+00,0.000000,8.290476e+08,0.612352,4.203634e+07,0.013106,0.000000e+00,0.000000,8.460000e+07
...,...,...,...,...,...,...,...,...,...
995,1.315493e+07,0.009174,1.764270e+08,0.172035,0.000000e+00,0.000000,0.000000e+00,0.000000,9.566877e+07
996,5.376487e+06,0.003400,5.414351e+08,0.451388,1.197198e+08,0.045439,0.000000e+00,0.000000,9.566877e+07
997,0.000000e+00,0.000000,3.056991e+07,0.026286,1.700519e+07,0.006849,0.000000e+00,0.000000,9.566877e+07
998,2.547146e+07,0.017398,8.453521e+06,0.008777,0.000000e+00,0.000000,1.083778e+08,0.104897,9.566877e+07


In [10]:
from ema_workbench import save_results
save_results(results, './output/' + CURRENT_DATE + '_results__policies_fixed__scenarios_generated')

[MainProcess/INFO] results saved successfully to /home/gabe/epa/epa1361_open/final assignment/output/2023-06-11T19:48:41_results__policies_fix__scenarios_generated
